<br>

## **Objective: To train simple CNN model and use trained model for classification**

<br>

## **Step 1: Import**

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as nnf
import torch.optim as optim
import torchvision.models as models
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
# for reproducibility

torch.manual_seed(42)

In [3]:
torch.set_printoptions(linewidth=300)

<br>

## **Step 2: Read and transform**

**Train dataset**

In [4]:
pf1 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_1')
pf2 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_2')
pf3 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_3')
pf4 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_4')
pf5 = pd.read_pickle('./data/cifar-10-batches-py/data_batch_5')

In [5]:
del pf1['batch_label']
del pf2['batch_label']
del pf3['batch_label']
del pf4['batch_label']
del pf5['batch_label']

In [6]:
pf1['data'] = list(pf1['data'])
pf2['data'] = list(pf2['data'])
pf3['data'] = list(pf3['data'])
pf4['data'] = list(pf4['data'])
pf5['data'] = list(pf5['data'])

In [7]:
pf1_df = pd.DataFrame(pf1)
pf2_df = pd.DataFrame(pf2)
pf3_df = pd.DataFrame(pf3)
pf4_df = pd.DataFrame(pf4)
pf5_df = pd.DataFrame(pf5)

In [8]:
train_df = pd.concat([pf1_df,
                      pf2_df,
                      pf3_df,
                      pf4_df,
                      pf5_df], ignore_index=True)

**Test dataset**

In [9]:
pf = pd.read_pickle('./data/cifar-10-batches-py/test_batch')

In [10]:
del pf['batch_label']

In [11]:
pf['data'] = list(pf['data'])

In [12]:
test_df = pd.DataFrame(pf)

<br>

## **Step 3: Transform to X_train and y_train in tensor form**

In [13]:
X_train = train_df['data'].tolist()

In [14]:
X_train = np.array(X_train)

In [15]:
X_train = torch.tensor(X_train, dtype=torch.float32).reshape((-1,3,32,32))

In [16]:
y_train = train_df['labels'].tolist()

In [17]:
y_train = np.array(y_train)

In [18]:
y_train = torch.tensor(y_train, dtype=torch.int64)

<br>

## **Step 4: Transform to X_test and y_test in tensor form**

In [19]:
X_test = test_df['data'].tolist()

In [20]:
X_test = np.array(X_test)

In [21]:
X_test = torch.tensor(X_test, dtype=torch.float32).reshape((-1,3,32,32))

In [22]:
y_test = test_df['labels'].tolist()

In [23]:
y_test = np.array(y_test)

In [24]:
y_test = torch.tensor(y_test, dtype=torch.int64)

<br>

## **Step 5: Push X_train, y_train, X_test, and y_test to GPU**

In [25]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [26]:
X_train = X_train.to(device)

In [27]:
X_train[:3]

tensor([[[[ 59.,  43.,  50.,  ..., 158., 152., 148.],
          [ 16.,   0.,  18.,  ..., 123., 119., 122.],
          [ 25.,  16.,  49.,  ..., 118., 120., 109.],
          ...,
          [208., 201., 198.,  ..., 160.,  56.,  53.],
          [180., 173., 186.,  ..., 184.,  97.,  83.],
          [177., 168., 179.,  ..., 216., 151., 123.]],

         [[ 62.,  46.,  48.,  ..., 132., 125., 124.],
          [ 20.,   0.,   8.,  ...,  88.,  83.,  87.],
          [ 24.,   7.,  27.,  ...,  84.,  84.,  73.],
          ...,
          [170., 153., 161.,  ..., 133.,  31.,  34.],
          [139., 123., 144.,  ..., 148.,  62.,  53.],
          [144., 129., 142.,  ..., 184., 118.,  92.]],

         [[ 63.,  45.,  43.,  ..., 108., 102., 103.],
          [ 20.,   0.,   0.,  ...,  55.,  50.,  57.],
          [ 21.,   0.,   8.,  ...,  50.,  50.,  42.],
          ...,
          [ 96.,  34.,  26.,  ...,  70.,   7.,  20.],
          [ 96.,  42.,  30.,  ...,  94.,  34.,  34.],
          [116.,  94.,  87.,  ...

In [28]:
y_train = y_train.to(device)

In [29]:
y_train[:3]

tensor([6, 9, 9], device='cuda:0')

In [30]:
X_test = X_test.to(device)

In [31]:
X_test[:3]

tensor([[[[158., 159., 165.,  ..., 137., 126., 116.],
          [152., 151., 159.,  ..., 136., 125., 119.],
          [151., 151., 158.,  ..., 139., 130., 120.],
          ...,
          [ 68.,  42.,  31.,  ...,  38.,  13.,  40.],
          [ 61.,  49.,  35.,  ...,  26.,  29.,  20.],
          [ 54.,  56.,  45.,  ...,  24.,  34.,  21.]],

         [[112., 111., 116.,  ...,  95.,  91.,  85.],
          [112., 110., 114.,  ...,  95.,  91.,  88.],
          [110., 109., 111.,  ...,  98.,  95.,  89.],
          ...,
          [124., 100.,  88.,  ...,  97.,  64.,  85.],
          [116., 102.,  85.,  ...,  82.,  82.,  64.],
          [107., 105.,  89.,  ...,  77.,  84.,  67.]],

         [[ 49.,  47.,  51.,  ...,  36.,  36.,  33.],
          [ 51.,  40.,  45.,  ...,  31.,  32.,  34.],
          [ 47.,  33.,  36.,  ...,  34.,  34.,  33.],
          ...,
          [177., 148., 137.,  ..., 146., 108., 127.],
          [168., 148., 132.,  ..., 130., 126., 107.],
          [160., 149., 132.,  ...

In [32]:
y_test = y_test.to(device)

In [33]:
y_test[:3]

tensor([3, 8, 8], device='cuda:0')

<br>

## **Step 6: Set up custom dataset (train and test) using torch.utils.data.Dataset**

In [34]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.y = y
        self.X = X
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        y = self.y[idx]
        X = self.X[idx]
        return X, y

In [35]:
train_dataset = CustomDataset(X_train, y_train)

In [36]:
test_dataset = CustomDataset(X_test, y_test)

<br>

## **Step 7: Set up custom dataloader (train and test) using torch.utils.data.DataLoader**

In [37]:
train_dataloader = DataLoader(train_dataset, batch_size=24, shuffle=False)

In [38]:
test_dataloader = DataLoader(test_dataset, batch_size=24, shuffle=False)

<br>

## **Step 8: Set up Convolutional Neural Network (CNN) model**

In [39]:
class cnn__model(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=9, kernel_size=(3,3), padding=1, stride=1)   # input = 3 x 32 x 32   output = 9 x 32 x 32
        self.pool = nn.MaxPool2d(kernel_size=(2,2))                                                     # input = 9 x 32 x 32   output = 9 x 16 x 16
        self.fc1 = nn.Linear(9*16*16, 100)                                                              # input = 9 x 16 x 16   output = 100
        self.fc2 = nn.Linear(100, 10)                                                                   # input = 100           output = 10

    def forward(self, x):
        x = self.pool(nnf.relu(self.conv1(x)))
        x = torch.flatten(x, 1)                                                                         # input = 9 x 16 x 16   output = 9 x 16 x 16
        x = nnf.relu(self.fc1(x))
        x = self.fc2(x)
        return x

cnn_model = cnn__model().to(device)
print(cnn_model)

cnn__model(
  (conv1): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2304, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [40]:
next(cnn_model.parameters()).is_cuda

True

<br>

## **Step 9: Train CNN model**

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.000001, momentum=0.9)

In [42]:
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss = running_loss + loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss}")

print()
print('Finished Training')

Epoch 1, Loss: 5459.909855127335
Epoch 2, Loss: 4387.320788502693
Epoch 3, Loss: 4192.974305033684
Epoch 4, Loss: 4078.79383790493
Epoch 5, Loss: 3992.896406173706
Epoch 6, Loss: 3922.212301015854
Epoch 7, Loss: 3862.294492483139
Epoch 8, Loss: 3809.947520852089
Epoch 9, Loss: 3763.2677326202393
Epoch 10, Loss: 3721.355789065361
Epoch 11, Loss: 3683.0649741888046
Epoch 12, Loss: 3647.940400004387
Epoch 13, Loss: 3615.2455648183823
Epoch 14, Loss: 3584.7410439252853
Epoch 15, Loss: 3555.881305336952
Epoch 16, Loss: 3528.643193244934
Epoch 17, Loss: 3502.5552400946617
Epoch 18, Loss: 3477.551467061043
Epoch 19, Loss: 3453.6456122994423
Epoch 20, Loss: 3430.5904825925827
Epoch 21, Loss: 3408.463115334511
Epoch 22, Loss: 3387.238999903202
Epoch 23, Loss: 3366.7186497449875
Epoch 24, Loss: 3346.9531366825104
Epoch 25, Loss: 3327.974506020546
Epoch 26, Loss: 3309.731041133404
Epoch 27, Loss: 3292.022572696209
Epoch 28, Loss: 3274.816314160824
Epoch 29, Loss: 3258.0716821551323
Epoch 30, Loss

**Save trained model for use later**

In [43]:
torch.save(cnn_model, 'trained_cnn_model')

**Display trained model at high level**

In [44]:
for i in cnn_model.state_dict():
    print(i, "\t", cnn_model.state_dict()[i].size())

conv1.weight 	 torch.Size([9, 3, 3, 3])
conv1.bias 	 torch.Size([9])
fc1.weight 	 torch.Size([100, 2304])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([10, 100])
fc2.bias 	 torch.Size([10])


**Display trained model at low level**

In [45]:
for i in cnn_model.state_dict():
    print(i, "\t", cnn_model.state_dict()[i])

conv1.weight 	 tensor([[[[ 0.1181,  0.1192, -0.0612],
          [ 0.1412, -0.0890, -0.0057],
          [-0.0852,  0.0963,  0.1388]],

         [[-0.1616,  0.1376,  0.0259],
          [ 0.1096, -0.0147,  0.0501],
          [-0.0273,  0.1266, -0.0075]],

         [[-0.0902,  0.0423, -0.0808],
          [-0.0276, -0.0875,  0.1123],
          [-0.1353, -0.0887, -0.0702]]],


        [[[-0.1148,  0.0483, -0.1807],
          [ 0.1795, -0.1656,  0.1354],
          [ 0.0133, -0.0787,  0.0932]],

         [[ 0.0189,  0.1672,  0.0135],
          [-0.0602,  0.0392, -0.0724],
          [ 0.0563,  0.1437,  0.0759]],

         [[-0.0980,  0.1196,  0.0250],
          [ 0.0991, -0.1257, -0.2068],
          [-0.0929, -0.1680,  0.1281]]],


        [[[ 0.0466,  0.0819,  0.0629],
          [-0.0106,  0.1462, -0.1405],
          [ 0.0107, -0.1419,  0.0471]],

         [[-0.0683,  0.0642, -0.0404],
          [ 0.1599, -0.1129, -0.1178],
          [-0.1096,  0.1680,  0.0545]],

         [[ 0.1839, -0.1529, 

<br>

## **Step 10: Use trained CNN model to predict (for comparison to Step 11)**

**Use trained model for classification**

In [46]:
# get actual labels for comparison

dataiter = iter(test_dataloader)

In [47]:
correct = 0
total = 0
i = 1
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)   # class with highest energy choosen as prediction
        print('---------- outputs ----------')
        print(torch.round(outputs.data, decimals=2))
        print('---------- highest energy ----------')
        print(torch.max(outputs.data, 1))
        print('---------- predicted ----------')
        print(predicted.data)
        images, labels = next(dataiter)
        print('---------- actual ----------')
        print(labels.data)
        total = total + labels.size(0)
        print('---------- total ----------')
        print(total)
        correct = correct + (predicted == labels).sum().item()
        print('---------- correct ----------')
        print(correct)
        print()
        print('--------------------------------------------------------------------------------')
        print()
        i = i + 1
        if i == 3:   # test on first 2 batches
            break

print()
print(f'Accuracy: {100 * correct / total} %')

---------- outputs ----------
tensor([[-1.2500, -0.3900,  1.1400,  1.7300,  0.4700,  1.6300,  1.3400, -2.6300, -0.3000, -2.0300],
        [ 4.1600,  5.9700, -0.9300, -0.5600, -3.0200, -3.4200, -4.0400, -2.7100,  5.3300,  5.9300],
        [ 2.4100,  2.2900, -0.9900,  1.1800, -1.1600, -1.4100, -4.5600, -2.0700,  2.4200,  3.8700],
        [ 2.4500,  2.1300,  0.6400, -0.2900, -0.5600, -1.2500, -0.5600,  0.4900,  3.5800,  1.0900],
        [ 0.0600, -0.8100,  3.0300,  2.0900,  4.5700,  2.2400,  3.8300,  2.0300, -0.2900, -1.3200],
        [-1.1500,  0.8900,  0.8500,  2.3200,  0.8300,  1.6300,  2.6300,  0.1700, -2.3200,  1.1800],
        [-0.7100,  5.0800,  0.7000,  4.0500, -1.8800,  0.2700, -0.3100, -0.7400, -1.0700,  3.7900],
        [ 0.4400, -0.4700,  2.6900,  2.4100,  2.1000,  0.7600,  4.0800,  1.0000, -1.6600, -1.5000],
        [ 1.1100, -1.2700,  2.7700,  2.9500,  2.5600,  3.3700,  0.7000,  1.6800, -1.7400, -2.2600],
        [ 3.7000,  7.3900,  0.3100,  0.8400, -2.6000, -2.1200, -2.3100

<br>

## **Step 11: Use saved trained CNN model to predict (for comparison to Step 10)**

**Load saved trained model**

In [48]:
my_trained_cnn_model = torch.load('trained_cnn_model')

**Check saved trained model is correct at high level**

In [49]:
for i in my_trained_cnn_model.state_dict():
    print(i, "\t", my_trained_cnn_model.state_dict()[i].size())

conv1.weight 	 torch.Size([9, 3, 3, 3])
conv1.bias 	 torch.Size([9])
fc1.weight 	 torch.Size([100, 2304])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([10, 100])
fc2.bias 	 torch.Size([10])


**Check saved trained model is correct at low level**

In [50]:
for i in my_trained_cnn_model.state_dict():
    print(i, "\t", my_trained_cnn_model.state_dict()[i])

conv1.weight 	 tensor([[[[ 0.1181,  0.1192, -0.0612],
          [ 0.1412, -0.0890, -0.0057],
          [-0.0852,  0.0963,  0.1388]],

         [[-0.1616,  0.1376,  0.0259],
          [ 0.1096, -0.0147,  0.0501],
          [-0.0273,  0.1266, -0.0075]],

         [[-0.0902,  0.0423, -0.0808],
          [-0.0276, -0.0875,  0.1123],
          [-0.1353, -0.0887, -0.0702]]],


        [[[-0.1148,  0.0483, -0.1807],
          [ 0.1795, -0.1656,  0.1354],
          [ 0.0133, -0.0787,  0.0932]],

         [[ 0.0189,  0.1672,  0.0135],
          [-0.0602,  0.0392, -0.0724],
          [ 0.0563,  0.1437,  0.0759]],

         [[-0.0980,  0.1196,  0.0250],
          [ 0.0991, -0.1257, -0.2068],
          [-0.0929, -0.1680,  0.1281]]],


        [[[ 0.0466,  0.0819,  0.0629],
          [-0.0106,  0.1462, -0.1405],
          [ 0.0107, -0.1419,  0.0471]],

         [[-0.0683,  0.0642, -0.0404],
          [ 0.1599, -0.1129, -0.1178],
          [-0.1096,  0.1680,  0.0545]],

         [[ 0.1839, -0.1529, 

**Use saved trained model for classification**

In [51]:
my_trained_cnn_model = my_trained_cnn_model.to(device)

In [52]:
next(my_trained_cnn_model.parameters()).is_cuda

True

In [53]:
# get actual labels for comparison

dataiter = iter(test_dataloader)

In [54]:
correct = 0
total = 0
i = 1
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = my_trained_cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)   # class with highest energy choosen as prediction
        print('---------- outputs ----------')
        print(torch.round(outputs.data, decimals=2))
        print('---------- highest energy ----------')
        print(torch.max(outputs.data, 1))
        print('---------- predicted ----------')
        print(predicted.data)
        images, labels = next(dataiter)
        print('---------- actual ----------')
        print(labels.data)
        total = total + labels.size(0)
        print('---------- total ----------')
        print(total)
        correct = correct + (predicted == labels).sum().item()
        print('---------- correct ----------')
        print(correct)
        print()
        print('--------------------------------------------------------------------------------')
        print()
        i = i + 1
        if i == 3:   # test on first 2 batches
            break

print()
print(f'Accuracy: {100 * correct / total} %')

---------- outputs ----------
tensor([[-1.2500, -0.3900,  1.1400,  1.7300,  0.4700,  1.6300,  1.3400, -2.6300, -0.3000, -2.0300],
        [ 4.1600,  5.9700, -0.9300, -0.5600, -3.0200, -3.4200, -4.0400, -2.7100,  5.3300,  5.9300],
        [ 2.4100,  2.2900, -0.9900,  1.1800, -1.1600, -1.4100, -4.5600, -2.0700,  2.4200,  3.8700],
        [ 2.4500,  2.1300,  0.6400, -0.2900, -0.5600, -1.2500, -0.5600,  0.4900,  3.5800,  1.0900],
        [ 0.0600, -0.8100,  3.0300,  2.0900,  4.5700,  2.2400,  3.8300,  2.0300, -0.2900, -1.3200],
        [-1.1500,  0.8900,  0.8500,  2.3200,  0.8300,  1.6300,  2.6300,  0.1700, -2.3200,  1.1800],
        [-0.7100,  5.0800,  0.7000,  4.0500, -1.8800,  0.2700, -0.3100, -0.7400, -1.0700,  3.7900],
        [ 0.4400, -0.4700,  2.6900,  2.4100,  2.1000,  0.7600,  4.0800,  1.0000, -1.6600, -1.5000],
        [ 1.1100, -1.2700,  2.7700,  2.9500,  2.5600,  3.3700,  0.7000,  1.6800, -1.7400, -2.2600],
        [ 3.7000,  7.3900,  0.3100,  0.8400, -2.6000, -2.1200, -2.3100